In [1]:
from pedect.config.BasicConfig import *
from pedect.predictor.GroundTruthPredictor import GroundTruthPredictor
from pedect.trainer.YoloTrainer import YoloTrainer
from pedect.dataset.CaltechDataset import CaltechDataset
from pedect.predictor.YoloPredictor import YoloPredictor
from tracker import re3_tracker 
from tqdm import tqdm

tracker = re3_tracker.Re3Tracker()
config = getConfigFromTrainId(2)

# trainer = YoloTrainer(config)
# trainer.train()

dataset = CaltechDataset(CALTECH_DIR)
gtPredictor = GroundTruthPredictor(dataset, "set01", "V005")
videoHolder = gtPredictor
pred = YoloPredictor(videoHolder, config)

Using TensorFlow backend.


Restoring:
re3/conv1/W_conv -> 	[11, 11, 3, 96] = 0MB
re3/conv1/b_conv -> 	[96] = 0MB
re3/conv1_skip/W_conv -> 	[1, 1, 96, 16] = 0MB
re3/conv1_skip/b_conv -> 	[16] = 0MB
re3/conv1_skip/prelu -> 	[16] = 0MB
re3/conv2/W_conv -> 	[5, 5, 48, 256] = 1MB
re3/conv2/b_conv -> 	[256] = 0MB
re3/conv2_skip/W_conv -> 	[1, 1, 256, 32] = 0MB
re3/conv2_skip/b_conv -> 	[32] = 0MB
re3/conv2_skip/prelu -> 	[32] = 0MB
re3/conv3/W_conv -> 	[3, 3, 256, 384] = 3MB
re3/conv3/b_conv -> 	[384] = 0MB
re3/conv4/W_conv -> 	[3, 3, 192, 384] = 2MB
re3/conv4/b_conv -> 	[384] = 0MB
re3/conv5/W_conv -> 	[3, 3, 192, 256] = 1MB
re3/conv5/b_conv -> 	[256] = 0MB
re3/conv5_skip/W_conv -> 	[1, 1, 256, 64] = 0MB
re3/conv5_skip/b_conv -> 	[64] = 0MB
re3/conv5_skip/prelu -> 	[64] = 0MB
re3/fc6/W_fc -> 	[74208, 1024] = 303MB
re3/fc6/b_fc -> 	[1024] = 0MB
re3/fc_output/W_fc -> 	[512, 4] = 0MB
re3/fc_output/b_fc -> 	[4] = 0MB
re3/lstm1/rnn/lstm_cell/bias -> 	[2048] = 0MB
re3/lstm1/rnn/lstm_cell/kernel -> 	[1536, 2048] = 12MB
re3/

In [2]:
n = gtPredictor.getLength()

rez = [len(pred.predictForFrame(i)) for i in tqdm(range(n))]
rezGT = [len(gtPredictor.predictForFrame(i)) for i in tqdm(range(n))]

100%|███████████████████████████████████████████████████████████████████████████| 1814/1814 [00:00<00:00, 67358.99it/s]


In [3]:
from pedect.evaluator.Evaluator import Evaluator

evaluator = Evaluator(config, pred, gtPredictor)
print(evaluator.evaluate(True))

100%|█████████████████████████████████████████████████████████████████████████████| 1814/1814 [00:07<00:00, 250.65it/s]


17.1
17.1


In [4]:
evaluator = Evaluator(config, pred, gtPredictor, 30)
print(evaluator.evaluate(True))

100%|█████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 235.59it/s]


19.17
19.17


In [5]:
from pedect.predictor.TrackerPredictor import TrackerPredictor
predictor = TrackerPredictor(pred, gtPredictor, tracker, BasicConfig())

In [6]:
from pedect.evaluator.HyperParametersTuner import HyperParametersTuner


noIterations = 30
ctRange = (0.0, 1.0)
rtRange = (0.0, 1.0)
stRange = (0.0, 1.0)
smpRange = (0.0, 1.0)
bestConfig, result = HyperParametersTuner.findBestConfig(predictor, gtPredictor, 100, noIterations, ctRange, rtRange, stRange, 
                                                     smpRange)
print(bestConfig, result)
    


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

13.57


  3%|██▊                                                                                | 1/30 [00:06<03:02,  6.30s/it]

14.57


  7%|█████▌                                                                             | 2/30 [00:12<02:52,  6.18s/it]

...

17.34


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [03:30<00:06,  6.08s/it]

17.34


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [03:36<00:00,  6.31s/it]


possibleLabels = {'people': (255, 0, 0), 'person-fa': (0, 0, 255), 'person': (0, 255, 0)}
trainId = 2
modelName = trained_weights_final.h5
inputShape = (416, 416)
freezeNoEpochs = 10
noFreezeNoEpochs = 10
isTiny = False
validationSplit = 0.1
freezeBatchSize = 16
noFreezeBatchSize = 16
loadPretrained = True
checkpointPeriod = 150
createThreshold = 0.29684617462238083
removeThreshold = 0.14902800913254022
surviveThreshold = 0.029460662950709815
surviveMovePercent = 0.049080990311648365
maxAge = 100
 17.34


In [11]:
from pedect.utils.demo import *

results = playVideo(predictor, videoHolder, tracker, bestConfig)

KeyboardInterrupt: 

In [10]:
evaluator = Evaluator(bestConfig, predictor, gtPredictor)
print(evaluator.evaluate(True))

100%|██████████████████████████████████████████████████████████████████████████████| 1814/1814 [01:54<00:00, 15.78it/s]


16.42
16.42


In [5]:
a = [rez[i] - rezGT[i] for i in range(len(rez))]
b = [x for x in a if x > 0]
c = [x for x in a if x < 0]
print(sum(b))
print(sum(c))

324
-544


In [1]:
from pedect.converter.ConverterToImages import *
from pedect.config.BasicConfig import BasicConfig
from pedect.utils.detector.debug import debug
from pedect.utils.detector.train import train
from pedect.utils.detector.evaluate import evaluate
import os


Using TensorFlow backend.


In [2]:
convertVideosToImages([("caltech" , "set01", "V000")], YOLO)
# convertVideosToImages([("caltech" , "set01", "V000"), ("caltech", "set02", "V000")], YOLO)

Saving images from caltech set set01 video V000


100%|██████████████████████████████████████████████████████████████████████████████| 1711/1711 [01:03<00:00, 26.84it/s]


In [9]:
# train(config)

In [11]:
# evaluate(config)

In [2]:
from pedect.utils.constants import *
from pedect.dataset.CaltechDataset import CaltechDataset
from pedect.predictor.FakePredictor import FakePredictor
from pedect.predictor.GroundTruthPredictor import GroundTruthPredictor
from pedect.predictor.TrackerPredictor import TrackerPredictor
from pedect.config.BasicConfig import BasicConfig
chosenDataset = CaltechDataset(CALTECH_DIR)
gtPredictor = GroundTruthPredictor(chosenDataset, "set01", "V000")
fakePredictor = FakePredictor((0.0, 0.8), (0.3, 1.0), gtPredictor, gtPredictor)
predictor = TrackerPredictor(fakePredictor, gtPredictor, tracker, BasicConfig())

Format mjpeg detected only with low score of 25, misdetection possible!


In [4]:
print(bestConfig, result)


        BasicConfig
        Possible labels = {'people': (255, 0, 0), 'person-fa': (0, 0, 255), 'person': (0, 255, 0)}
        Batch size = 1
        Steps per epoch = 100
        Number of epochs = 100
        Image max side = 600
        Backbone = mobilenet128_1.0
        Create Threshold = 0.347968
        Remove Threshold = 0.161871
        Survive Threshold = 0.914376
        Survive Move Percent = 0.840482
        Max Age = 100
         69.75
